In [1]:
 import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    # plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    # plt.legend([string, 'val_'+string])
    plt.show()

In [3]:
df = pd.read_csv('oversampled_CICIDS2018_Thursday.csv')
df.head()

,Unnamed: 0,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,target
0,45026,80,6,15/02/2018 09:52:24,57871410,2,0,0,0,0,...,0.000000,0.000000,0,0,5.787141e+07,0.000000e+00,57871410,57871410,DoS attacks-GoldenEye,1
1,27221,80,6,15/02/2018 09:49:24,4485538,5,3,459,972,459,...,0.000000,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,DoS attacks-GoldenEye,1
2,888204,443,6,15/02/2018 02:15:13,116039294,19,17,1109,594,517,...,63537.166667,36001.448059,177857,53084,9.606401e+06,1.393516e+06,10009077,5181394,Benign,0
3,361821,53037,6,15/02/2018 10:57:51,124,2,0,0,0,0,...,0.000000,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,Benign,0
4,859694,80,6,15/02/2018 12:08:17,38003752,7,8,434,1951,431,...,54212.666667,23938.964061,81855,40384,1.000524e+07,1.776541e+03,10006272,10003184,Benign,0


In [11]:
df2 = df.groupby(['Timestamp'])['Flow Duration'].count()
df2 = pd.DataFrame(df2).reset_index()
df2.columns=['Timestamp','BiFlowsCount']
df = df.merge(df2, left_on='Timestamp', right_on='Timestamp')
#df = df.sort_values(' Timestamp')

df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Target,BiFlowsCount
0,80,6,15/02/2018 09:52:24,57871410,2,0,0,0,0,0,...,0.0,0,0,57871410.0,0.0,57871410,57871410,DoS attacks-GoldenEye,1,1466
1,80,6,15/02/2018 09:52:24,57718904,2,0,0,0,0,0,...,0.0,0,0,57718904.0,0.0,57718904,57718904,DoS attacks-GoldenEye,1,1466
2,80,6,15/02/2018 09:52:24,1845282,2,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye,1,1466
3,80,6,15/02/2018 09:52:24,1837481,2,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye,1,1466
4,80,6,15/02/2018 09:52:24,57879663,2,0,0,0,0,0,...,0.0,0,0,57879663.0,0.0,57879663,57879663,DoS attacks-GoldenEye,1,1466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,52058,6,15/02/2018 02:35:59,30526349,2,0,112,0,64,48,...,0.0,0,0,30526349.0,0.0,30526349,30526349,Benign,0,1
999996,80,6,15/02/2018 05:37:54,846,3,4,161,488,161,0,...,0.0,0,0,0.0,0.0,0,0,Benign,0,1
999997,80,6,15/02/2018 03:54:27,146,2,0,0,0,0,0,...,0.0,0,0,0.0,0.0,0,0,Benign,0,1
999998,3389,6,15/02/2018 12:48:28,5841635,11,7,1148,1581,677,0,...,0.0,0,0,0.0,0.0,0,0,Benign,0,1


In [14]:
#del df['Unnamed: 0']
#del df['target']

In [13]:
 features =['Fwd Seg Size Min', 'Init Fwd Win Byts', 'BiFlowsCount', 'Protocol', 'Dst Port', 'PSH Flag Cnt', 'Bwd Pkt Len Max', 'Bwd Pkt Len Std', 'ACK Flag Cnt',
            'Flow IAT Min', 'Init Bwd Win Byts', 'Flow IAT Mean', 'Fwd IAT Tot', 'Fwd IAT Min', 'Fwd IAT Max', 'Fwd IAT Mean', 'Idle Mean', 
            'Flow Duration', 'Flow IAT Max', 'Pkt Len Max' ]

In [15]:
fraction = 0.5
df = df.sample(frac=fraction, replace=True, random_state=1)

In [16]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [17]:
df['Target']=df['Label'].apply(targetify)

In [18]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)

C:\Users\kdagi\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
y = df['Target']

In [20]:
X = X.fillna(0)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(400000, 20) (400000,)
(100000, 20) (100000,)


In [22]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 64), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
]) 

In [24]:
model.compile(loss='binary_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [25]:
history = model.fit(X_train.values, y_train.values, epochs=3)


Train on 400000 samples
Epoch 1/3
400000/400000 [==============================] - 1879s 5ms/sample - loss: 0.5089 - accuracy: 0.7140
Epoch 2/3
400000/400000 [==============================] - 1604s 4ms/sample - loss: 0.4970 - accuracy: 0.7225
Epoch 3/3
400000/400000 [==============================] - 1802s 5ms/sample - loss: 0.4968 - accuracy: 0.7226


In [26]:
pred_class = model.predict(X_test.values[:])
print(pred_class)

[[0.55424243]
 [0.718163  ]
 [0.55424243]
 ...
 [0.55424243]
 [0.718163  ]
 [0.55424243]]


In [41]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [28]:
print(round(pred_class[2][0]))
print(round(pred_class[1][0]))

1.0
1.0


In [29]:
predictions = [int(round(x[0])) for x in pred_class]
true_class = list(y_test)

In [30]:
print(true_class[:10], len(true_class))
print(predictions[:10], len(predictions))

[1, 1, 0, 0, 0, 1, 1, 0, 0, 0] 100000
[1, 1, 1, 0, 0, 1, 1, 1, 1, 1] 100000


In [31]:
compdf = pd.DataFrame({'pred_class':predictions, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))

       pred_class  true_class
0               1           1
62500           1           1
62511           1           1
62510           1           0
62509           1           1
62506           1           1
62505           1           0
62504           1           0
62502           1           1
62501           1           1


In [32]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])

In [33]:
print(predictions[:10])
print(true_class[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 0, 0, 1, 1]


In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confm = confusion_matrix(true_class, predictions)
confm

array([[22446, 27551],
       [  112, 49891]], dtype=int64)

In [36]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.72337

In [37]:
print("False alarm rate is")
fp/(tn+fp)

False alarm rate is


0.551053063183791

In [38]:
print("Precision is")
Precision = tp/(tp+fp)
Precision

Precision is


0.6442369773507916

In [39]:
print("Recall is")
Recall = tp/(tp+fn)
Recall

Recall is


0.9977601343919364

In [40]:
print("F1 is")
2*Precision*Recall/(Precision+Recall)

F1 is


0.7829416611087135